In [ ]:
"""进程和线程

单核CPU以时间片轮转的方式来执行多任务，真正的并行执行多任务需要在多核CPU上实现
一个进程Process至少包含一个线程Thread（MainThread），线程是最小的执行单元
多任务实现：多进程模式 多线程模式 多进程+多线程模式
调度进程和线程，完全由操作系统决定，程序自己不能决定什么时候执行，执行多长时间
在Unix/Linux下，可以使用os.fork()调用实现多进程，跨平台可以使用multiprocessing模块
使用进程池的方式批量创建子进程
进程间通信通过Queue Pipes等实现
任何进程默认都会启动一个线程，称为主线程MainThread，主线程又可以启动新的线程
在多进程中，同一个变量在每个进程中有一份拷贝，互不影响；在多线程中，变量在线程间共享
使用线程锁threading.Lock()来对线程进行隔离，避免冲突
Python进程有各自独立的GIL锁，互不影响，因此可以使用多进程来实现多核任务，但无法使用多线程来实现
ThreadLocal变量虽然是全局变量，但每个线程都只能读写自己线程的独立副本，互不干扰
采用多任务需要考虑的因素：任务数量（线程切换）和任务类型（计算密集型 vs IO密集型）
"""

In [ ]:
"""进程的基本用法"""

from multiprocessing import Process
import os
import time


# 进程执行函数
def run_proc(name):
    print('Child process %s (%s)' % (name, os.getpid()))
    time.sleep(5)


if __name__ == '__main__':
    print('Parent process %s' % os.getpid())
    
    # 创建子进程，传入执行函数和函数参数
    p = Process(target=run_proc, args=('test_process',))
    print('Child process start')
    p.start()
    
    # 等待子进程结束后再继续往下执行
    p.join()
    print('Child process end')

In [6]:
"""进程队列"""

from multiprocessing import Process, Queue
import os
import time
import random


# 写数据执行函数
def write_data(q):
    print('Process to write: %s' % os.getpid())
    for d in ['A', 'B', 'C', 'D']:
        print('Put %s to queue' % d)
        q.put(d)
        time.sleep(random.random())


# 读数据执行函数
def read_data(q):
    print('Process to read: %s' % os.getpid())
    while True:
        d = q.get(True)
        print('Get %s from queue' % d)


if __name__ == '__main__':
    
    # 父进程创建Queue，并传给各个子进程
    q = Queue()
    pw = Process(target=write_data, args=(q,))
    pr = Process(target=read_data, args=(q,))
    
    # 启动子进程
    pw.start()
    pr.start()
    
    # 等待写数据进程结束
    pw.join()
    
    # 强制终止读进程
    pr.terminate()

In [ ]:
"""进程池"""

from multiprocessing import Pool
import os
import time
import random


# 进程执行函数
def proc_task(name):
    print('Run task %s (%s) ...' % (name, os.getpid()))
    start = time.time()
    time.sleep(random.random() * 5)
    end = time.time()
    print('Task %s runs %0.2f seconds' % (name, (end - start)))
    

if __name__ == '__main__':
    print('Parent process %s' % os.getpid())
    
    p = Pool(5)
    for i in range(5):
        p.apply_async(proc_task, args=(i,))
    
    print('Waiting for all sub processing done ...')
    p.close()
    p.join()
    print('All sub processing done')

Parent process 2984
Waiting for all sub processing done ...


In [ ]:
"""线程的基本使用"""

import threading
import time


# 线程执行函数
def run_thread():
    print('Thread %s is running ...' % threading.current_thread().name)
    n = 0
    while n < 5:
        n = n + 1
        print('Thread %s >>> %s' % (threading.current_thread().name, n))
        time.sleep(1)
    print('Thread %s ended' % threading.current_thread().name)
    

# 启动线程
print('Thread %s is running ...' % threading.current_thread().name)

t = threading.Thread(target=run_thread, name='TestThread')
t.start()
t.join()

print('Thread %s ended' % threading.current_thread().name)

In [3]:
"""多线程的基本使用

threading库可以在单独的线程中执行任何可调用的对象
创建好的线程不会立即执行，除非调用它的start()方法
"""

from threading import Thread
import time

# 目标函数
def countdown(n):
    while n > 0:
        print('Countdown: ', n)
        n -= 1
        time.sleep(1)
        
# 启动线程
t = Thread(target=countdown, args=(10,))
t.start()

# 判断线程的执行状态
if t.is_alive():
    print('Running...')
else:
    print('Completed!')
    
# 后台执行需要长时间运行的线程
# t = Thread(target=countdown, args=(10,), daemon=True)

Countdown: Running... 10

Countdown:  9
Countdown:  8
Countdown:  7
Countdown:  6
Countdown:  5
Countdown:  4
Countdown:  3
Countdown:  2
Countdown:  1
